In [1]:
from datetime import date, datetime
import bisect

class Airline:
    def __init__(self, id_code, name):
        self.id = id_code  # two letters code, e.g. "AA"
        self.name = name
        self.planes = []  # list of Airplane instances

class Airplane:
    def __init__(self, id_num, current_location, company):
        self.id = id_num  # int
        self.current_location = current_location  # Airport instance
        self.company = company  # Airline instance
        self.next_flights = []  # list of Flight instances sorted by date

        # Register this plane with airline and airport
        company.planes.append(self)
        current_location.planes.append(self)

    def fly(self, destination):
        # Find flight scheduled to this destination on the earliest date
        for flight in self.next_flights:
            if flight.destination == destination:
                flight.take_off()
                flight.land()
                # Remove this flight from next_flights after flying
                self.next_flights.remove(flight)
                return f"Plane {self.id} flew from {flight.origin.city} to {flight.destination.city} on {flight.date}"
        return f"No scheduled flight for destination {destination.city}."

    def location_on_date(self, query_date):
        # Returns airport where plane will be on that date
        last_location = self.current_location
        for flight in sorted(self.next_flights, key=lambda f: f.date):
            if flight.date > query_date:
                break
            last_location = flight.destination
        return last_location

    def available_on_date(self, query_date, location):
        # True if plane is at `location` on query_date and has no flight that day
        if self.location_on_date(query_date) != location:
            return False
        for flight in self.next_flights:
            if flight.date == query_date:
                return False
        return True

class Flight:
    def __init__(self, date_obj, destination, origin, plane):
        self.date = date_obj  # datetime.date
        self.destination = destination  # Airport instance
        self.origin = origin  # Airport instance
        self.plane = plane  # Airplane instance

        # Generate ID: destination code + airline code + date in YYYYMMDD
        self.id = f"{destination.city}{plane.company.id}{date_obj.strftime('%Y%m%d')}"

        # Add flight to origin and destination schedules (keeping them sorted by date)
        bisect.insort(origin.scheduled_departures, self)
        bisect.insort(destination.scheduled_arrivals, self)

        # Add flight to plane's schedule (keep sorted)
        bisect.insort(plane.next_flights, self)

    def take_off(self):
        # Remove plane from origin airport planes
        if self.plane in self.origin.planes:
            self.origin.planes.remove(self.plane)

    def land(self):
        # Add plane to destination airport planes and update plane location
        self.destination.planes.append(self.plane)
        self.plane.current_location = self.destination

    # For sorting flights by date
    def __lt__(self, other):
        return self.date < other.date

    def __repr__(self):
        return f"Flight {self.id} from {self.origin.city} to {self.destination.city} on {self.date}"

class Airport:
    def __init__(self, city_code):
        self.city = city_code
        self.planes = []  # planes currently at airport
        self.scheduled_departures = []  # future flights from airport (sorted by date)
        self.scheduled_arrivals = []  # future flights to airport (sorted by date)

    def schedule_flight(self, destination, date_obj):
        # Find available plane at this airport for that date
        for airline in set(plane.company for plane in self.planes):
            for plane in airline.planes:
                if plane.available_on_date(date_obj, self):
                    # Schedule flight with this plane
                    flight = Flight(date_obj, destination, self, plane)
                    return flight
        return None  # no plane available

    def info(self, start_date, end_date):
        print(f"Scheduled flights from {self.city} between {start_date} and {end_date}:")
        for flight in self.scheduled_departures:
            if start_date <= flight.date <= end_date:
                print(flight)

# --------------- TEST ---------------

if __name__ == "__main__":
    # Create airlines
    airline1 = Airline("AA", "American Airlines")
    airline2 = Airline("DL", "Delta Airlines")

    # Create airports
    jfk = Airport("JFK")
    lax = Airport("LAX")
    ord_airport = Airport("ORD")

    # Create airplanes
    plane1 = Airplane(1, jfk, airline1)
    plane2 = Airplane(2, lax, airline1)
    plane3 = Airplane(3, jfk, airline2)

    # Schedule flights
    f1 = jfk.schedule_flight(lax, date(2025, 7, 20))
    f2 = lax.schedule_flight(jfk, date(2025, 7, 21))
    f3 = jfk.schedule_flight(ord_airport, date(2025, 7, 22))

    # Show info on JFK flights between dates
    jfk.info(date(2025, 7, 19), date(2025, 7, 23))

    # Fly plane1 from JFK to LAX (if scheduled)
    print(plane1.fly(lax))

    # Where is plane1 on 2025-07-21?
    print(f"Plane1 location on 2025-07-21: {plane1.location_on_date(date(2025, 7, 21)).city}")

    # Check availability
    print(f"Plane1 available on 2025-07-22 at LAX? {plane1.available_on_date(date(2025,7,22), lax)}")
    print(f"Plane1 available on 2025-07-22 at JFK? {plane1.available_on_date(date(2025,7,22), jfk)}")


Scheduled flights from JFK between 2025-07-19 and 2025-07-23:
Flight LAXAA20250720 from JFK to LAX on 2025-07-20
Flight ORDAA20250722 from JFK to ORD on 2025-07-22
Plane 1 flew from JFK to LAX on 2025-07-20
Plane1 location on 2025-07-21: JFK
Plane1 available on 2025-07-22 at LAX? False
Plane1 available on 2025-07-22 at JFK? False
